<a href="https://colab.research.google.com/github/rmwvamp/ECGNotebooks/blob/main/GSRFeatureExtractionusing_pyEDA_25GBRAM_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Drive Mounting and Dependencies

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
!pip install wfdb
!pip install opensignalsreader
!pip install pyhrv
!pip install biosppy
!pip install pyhrv
!pip install torchvision

     |████████████████████████████████| 137 kB 4.2 MB/s 
     |████████████████████████████████| 11.2 MB 41.1 MB/s 
     |████████████████████████████████| 930 kB 53.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 3.2 MB 4.3 MB/s 
     |████████████████████████████████| 1.5 MB 45.1 MB/s 
     |████████████████████████████████| 230 kB 50.2 MB/s 
     |████████████████████████████████| 47 kB 4.0 MB/s 
  Created wheel for biosppy: filename=biosppy-0.8.0-py2.py3-none-any.whl size=100176 sha256=171be954f84ed6d145d007cd3afb8d8359afd2f82cc2cb6286f4e474ebbd61fd
  Stored in directory: /root/.cache/pip/wheels/32/17/9d/77adbf0db713546eaa1c7a1480b2dce91fa724a133115dc20c
  Created wheel for spectrum: filename=spectrum-0.8.1-cp37-cp37m-linux_x86_64.whl size=235155 sha256=755cea4011c64c9f2969e40c8f738ca054befce3df3ed07beb2241823ffe44b5
  Stored in directory: /root/.cache/pip/wheels/79/db/9c/92fa684ca088447807d08672e7609b48102c6161ac9c7e3c62
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64232 sha256=d8bd240acdedd9be47031345dc578e266d0f61df483e2ea979e6dde30dab7af0
  Stored in directory: /r

In [ ]:
import os
import sys
import csv

#median
import statistics
from statistics import median

import os
from glob import glob

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

# physionet data
import wfdb
from wfdb import processing

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',100000)


### Cloning pyEDA: An Open-Source Python Toolkit for Pre-processing and Feature Extraction of Electrodermal Activity and Importing


In [ ]:
%cd /gdrive/MyDrive/SamsungWork

/gdrive/MyDrive/SamsungWork


In [ ]:
!ls

ECGExtractedFeatures_25April22		 ECGFeatures5Seconds_2_CompleteDF
ECGFeatures10Seconds			 ECGFeatures5SecondsCompleteDF
ECGFeatures10Seconds_1May22		 ExtractedFeatures
ECGFeatures10Seconds_2May22		 FinalECGFeatures10Seconds
ECGFeatures10Seconds_3May22		 FinalECGFeatures5SecondsCompleteDF
ECGFeatures5Seconds			 Participant2Data
ECGFeatures5Seconds_1			 pyEDA
ECGFeatures5Seconds_2_60SecondWindowing


In [ ]:
# ! git clone https://github.com/HealthSciTech/pyEDA

In [ ]:
!ls

ECGExtractedFeatures_25April22		 ECGFeatures5Seconds_2_CompleteDF
ECGFeatures10Seconds			 ECGFeatures5SecondsCompleteDF
ECGFeatures10Seconds_1May22		 ExtractedFeatures
ECGFeatures10Seconds_2May22		 FinalECGFeatures10Seconds
ECGFeatures10Seconds_3May22		 FinalECGFeatures5SecondsCompleteDF
ECGFeatures5Seconds			 Participant2Data
ECGFeatures5Seconds_1			 pyEDA
ECGFeatures5Seconds_2_60SecondWindowing


In [ ]:
from pyEDA.main import *

# Setting up Files

## Defining Process Statistical Feature GSR

In [ ]:
def segmentwise_custom(gsrdata, sample_rate, segment_width=120, segment_overlap=0,
                        segment_min_size=5):
	'''processes passed gsrdata.
	Processes the passed gsr data. Returns measures{} dict containing results.
	
	Parameters
	----------
	gsrdata : 1d array or list 
		array or list containing gsr data to be analysed
	sample_rate : int or float
		the sample rate with which the gsr data is sampled
	segment_width : int or float
		width of segments in seconds
		default : 120
	segment_overlap: float
		overlap fraction of adjacent segments.
		Needs to be 0 <= segment_overlap < 1.
		default : 0 (no overlap)
	segment_min_size : int
		often a tail end of the data remains after segmenting into segments.
		default : 20
	
	Returns
	-------
	gsrdata_segmentwise : 2d array or list 
		array or list containing segmentwised gsr data to be analysed
	orking_data : dict
		dictionary object used to store temporary values.
	s_measures : dict
		dictionary object used by heartpy to store computed measures.
	'''
	slice_indices = make_windows(gsrdata, sample_rate, segment_width, segment_overlap, segment_min_size)
	
	s_measures = {}
	s_working_data = {}
	
	gsrdata_segmentwise = []
	for i, ii in slice_indices:
		gsrdata_segmentwise.append(gsrdata[i:ii])
		s_measures = append_dict(s_measures, 'segment_indices', (i, ii))
		s_working_data = append_dict(s_working_data, 'segment_indices', (i, ii))
	return s_working_data, s_measures, gsrdata_segmentwise

In [ ]:
def rolling_mean_custom(data, windowsize, sample_rate):
  '''calculates rolling mean
    Function to calculate moving average over the passed data
	
    Parameters
    ----------
    data : 1-d array
        array containing the gsr data
    windowsize : int or float 
        the moving average window size in seconds 
    sample_rate : int or float
        the sample rate of the data set
		
    Returns
    -------
    rol_mean : 1-d array
        array containing computed rolling mean
  '''
  avg_hr = (np.mean(data))
  data_arr = np.array(data)
	
  t_windowsize = int(windowsize*sample_rate)
  t_shape = data_arr.shape[:-1] + (data_arr.shape[-1] - t_windowsize + 1, t_windowsize)
  t_strides = data_arr.strides + (data_arr.strides[-1],)
  sep_win = np.lib.stride_tricks.as_strided(data_arr, shape=t_shape, strides=t_strides)
  rol_mean = np.mean(sep_win, axis=1)
	
  missing_vals = np.array([avg_hr for i in range(0, int(abs(len(data_arr) - len(rol_mean))/2))])
  rol_mean = np.insert(rol_mean, 0, missing_vals)
  rol_mean = np.append(rol_mean, missing_vals)

  #only to catch length errors that sometimes unexplicably occur. 
  ##Generally not executed, excluded from testing and coverage
  if len(rol_mean) != len(data): # pragma: no cover
    lendiff = len(rol_mean) - len(data)
    if lendiff < 0:
      rol_mean = np.append(rol_mean, 0)
    else:
      rol_mean = rol_mean[:-1]
	  
  return rol_mean

In [ ]:
def process_statistical_customfunction(gsr_signal, use_scipy=True, sample_rate=128, new_sample_rate=40, segment_width=600, segment_overlap=0):
	gsrdata = np.array(gsr_signal)
	

	#################################################################################
	############################## Preprocessing Part ###############################
	
	# Resample the data based on original data rate of your device, here: 128Hz
	# data = resample_data(gsrdata, sample_rate, new_sample_rate)
	
	# Segmentwise the data based on window sizes
	s_working_data, s_measures, gsrdata_segmentwise = segmentwise_custom(data, sample_rate=new_sample_rate, segment_width=segment_width, segment_overlap=segment_overlap)
	
	preprocessed_gsr = []
	for i in gsrdata_segmentwise:
		preprocessed_gsr.append(rolling_mean_custom(i, 1./new_sample_rate, new_sample_rate))
	
	############################## Preprocessing Part ###############################
	#################################################################################
	
	
	
	#################################################################################
	############################ Feature Extraction Part ############################
	
	# Statistical Feature Extraction
	for i in preprocessed_gsr:
		working_data, measures = statistical_feature_extraction(i, new_sample_rate, use_scipy=use_scipy)
		for k in measures.keys():
			s_measures = append_dict(s_measures, k, measures[k])
		for k in working_data.keys():
			s_working_data = append_dict(s_working_data, k, working_data[k])
	
	wd = s_working_data
	m = s_measures
		
	############################ Feature Extraction Part ############################
	#################################################################################
	
	return m, wd, preprocessed_gsr

### Setting up Baseline files

In [ ]:
%cd /gdrive/MyDrive/Class_Dataset/CLAS Dataset/Participants/Part10/by_block

/gdrive/.shortcut-targets-by-id/1oRmZOKPo1r153R8uQckeuhxEs5K25BXY/Class_Dataset/CLAS Dataset/Participants/Part10/by_block


In [ ]:
baseline_file=['1_gsr_ppg_.csv','4_gsr_ppg_.csv','7_gsr_ppg_.csv','10_gsr_ppg_.csv']

In [ ]:
baseline_df=pd.read_csv(baseline_file[0])

In [ ]:
for base in baseline_file[1:]:
    data=pd.read_csv(base)
    baseline_df=baseline_df.append(data,ignore_index=True)

In [ ]:
baseline_df=baseline_df.drop(['Timestamp','ppg','PythonTimestamp','accelx','accely','accelz'],axis=1)

In [ ]:
 baseline_df

,gsr
0,157.394536
1,157.394536
2,157.521110
3,157.647887
4,157.521110
...,...
38410,100.630619
38411,100.682344
38412,100.682344
38413,100.682344


In [ ]:
m, wd, eda_clean = process_statistical_customfunction(baseline_df, use_scipy=True, sample_rate=256, new_sample_rate=256, segment_width=20, segment_overlap=10)


In [ ]:
wd

In [ ]:
print(type(wd))

In [ ]:
print (wd.keys())

In [ ]:
print (wd.values())

In [ ]:
df = pd.DataFrame.from_dict(wd, orient = 'index')

In [ ]:
df

In [ ]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:            25G        1.0G         23G        1.2M        922M         24G
Swap:            0B          0B          0B
